In [ ]:
import numpy as np
class Game():

  def __init__(self):
    self.game_finished = False
    self.grid = [
                  [0, -1, 1],
                  [-1, 0, 0],
                  [0, 0, 0],
                ]
    self.all_state = [
                  [6, 7, 8],
                  [3, 4, 5],
                  [0, 1, 2],
                ]
    self.action_space = [0,1,2,3]
    self.agent()
    self.display()

  def reset(self):
    self.game_finished = False
    self.agent()

  def agent(self):
    self.line_init = 2
    self.col_init = 0
    self.reward = 0
    self.init_state = 0
    self.current_col = self.col_init
    self.current_line = self.line_init
    self.agent_position = [self.line_init, self.col_init]

  def step(self, action: int, is_displayed = False):
    if action in self.action_space:
        if action == 0:
            self.current_line = max(self.current_line - 1, 0)
        elif action == 1:
            self.current_line = min(self.current_line + 1, 2)
        elif action == 2:
            self.current_col = max(self.current_col - 1, 0)
        elif action == 3:
            self.current_col = min(self.current_col + 1, 2)

        self.agent_position = [self.current_line, self.current_col]
        self.state = self.all_state[self.agent_position[0]][self.agent_position[1]]
        self.get_reward()
        self.display(is_displayed)
        self.is_finished(is_displayed)
        agent_position = self.agent_position
        reward = self.reward
        state = self.state
    else:
       print("Action impossible")
    return agent_position, reward, state

  def display(self, is_displayed = True):
    if is_displayed:
      grid_displayed = self.grid
      grid_displayed[self.agent_position[0]][self.agent_position[1]] = "   X"
      for ligne in grid_displayed:
        for element in ligne:
            print("{:4}".format(element), end="")
        print()
      print("End of turn")
      grid_displayed[self.agent_position[0]][self.agent_position[1]] = self.reward


  def get_reward(self):
    self.reward = self.grid[self.current_line][self.current_col]

  def is_finished(self, is_displayed = False):
     if self.reward == 1:
      self.game_finished = True
      if is_displayed:
          print("End of Game")


In [ ]:
import random
import numpy as np
class player():

  def __init__(self):
    self.env = Game()

  def play(self, is_human, train, Q_predict = []):
    self.train = train
    self.Q_predict = Q_predict
    if is_human:
      self.play_is_human()
    else:
      self.play_non_human()

  def play_is_human(self):
    while not self.env.game_finished:
      action = int(input())
      agent_position, reward, state = self.env.step(action)
      print(state)

  def q_update(self):
      alpha = 0.1
      gamma = 0.9
      self.Q[self.hist["State"][-2]][self.hist["Action"][-1]] =  self.Q[self.hist["State"][-2]][self.hist["Action"][-1]] + alpha * (self.hist["Reward"][-1] + gamma * max(self.Q[self.hist["State"][-1]]) - self.Q[self.hist["State"][-2]][self.hist["Action"][-1]])

  def play_non_human(self, epoch = 100):
    eps = 0.1
    state = self.env.init_state
    if self.train:
      self.Q = [
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
      ]
      self.hist = {
          "Reward": [],
          "Action": [],
          "State": [state]
                     }
      for i in range(epoch):
        while not self.env.game_finished:
          if random.random() < eps:
            action = random.randint(0,len(self.env.action_space)- 1)
            agent_position, reward, state = self.env.step(action, is_displayed = False)
          else:
            list_q = self.Q[state]
            action = list_q.index(max(list_q))
            agent_position, reward, state = self.env.step(action,is_displayed = False)
          if self.train:
            self.hist["Reward"].append(reward)
            self.hist["Action"].append(action)
            self.hist["State"].append(state)
            self.q_update()
        if self.train:
          self.env.reset()
          self.hist = {
              "Reward": [],
              "Action": [],
              "State": [self.env.init_state]
          }
    else:
        tour = 0
        while not self.env.game_finished:
          list_q = self.Q_predict[state]
          action = list_q.index(max(list_q))
          agent_position, reward, state = self.env.step(action, is_displayed = True)
          tour += 1
          if tour == 10:
            break




In [ ]:
is_human = False
train = True

joueur_1 = player()
joueur_1.play(is_human, train)

   0  -1   1
  -1   0   0
   X   0   0
End of turn


In [ ]:
is_human = False
train = False
q_predict = joueur_1.Q
joueur_2 = player()
joueur_2.play(is_human, train, q_predict)

   0  -1   1
  -1   0   0
   X   0   0
End of turn
   0  -1   1
  -1   0   0
   0   X   0
End of turn
   0  -1   1
  -1   X   0
   0   0   0
End of turn
   0  -1   1
  -1   0   X
   0   0   0
End of turn
   0  -1   X
  -1   0   0
   0   0   0
End of turn
End of Game


In [ ]:
joueur_1.Q

[[-0.2969118260918732,
  0.01216811482629864,
  0.01874149742151831,
  0.1509435222636928],
 [0.47805105614851257, 0.0, 0.0, 0.0],
 [0.16824262316706234, 0, 0, 0],
 [0.0, 0.004740723985712286, -0.1, 0.8018811269732824],
 [-0.39356846352969993,
  0.026588515803668687,
  -0.4328861516745645,
  0.8984232476822366],
 [0.999823035654572,
  0.008573229097816392,
  0.05134745394536016,
  0.20469629044181467],
 [0.0, -0.8821798004241401, 0.0, -0.5006662477772864],
 [-0.2341441, 0.059760223051500014, 0.0, 0.8499053647030009],
 [0, 0, 0, 0]]